In [ ]:
import openai
import pandas as pd
from collections import defaultdict
import json
import os

from src.utils import call_gpt3_api, call_gpt4_api
from src.mcq_frq import *
from src.eval import eval_experiment
from tqdm.auto import tqdm

In [ ]:
# Set up OpenAI API credentials for GPT-3.5
openai_key = open("../keys/openai_key.txt", "r")
openai.api_key = openai_key.readlines()[0].strip()

organization_id = open("../keys/rajpurkarlab_org_id.txt", "r")
openai.organization = organization_id.readlines()[0].strip()

In [ ]:
# Read dataset
dataset = pd.read_csv("./data/dataset_final.tsv", sep="\t")

# Experiment

In [ ]:
def summarized_withPE_experiment(all_cases, summarized, case_idx, experiment_name, gpt_model_type):
    
    prompts = {'mcq_4': get_mcq_withPE_prompt, 'mcq_many': get_mcq_withPE_prompt, 'frq': get_frq_withPE_prompt}
    
    call_gpt_api = {"gpt-3.5": call_gpt3_api, "gpt-4": call_gpt4_api}
    
    age = all_cases.loc[case_idx, "age"]
    sex = all_cases.loc[case_idx, "sex"]
    pe = all_cases.loc[case_idx, "physical_exam"]
    
    if experiment_name == "mcq_4":
        choices = get_choices(all_cases, case_idx)
        prompt = prompts[experiment_name](summarized, age, sex, pe, choices)
    elif experiment_name == "mcq_many":
        choices = get_all_choices()
        prompt = prompts[experiment_name](summarized, age, sex, pe, choices)
    else:
        prompt = prompts[experiment_name](summarized, age, sex, pe)
        
    input_msg = [{"role":"system", "content": prompt}]
    response = call_gpt_api[gpt_model_type](input_msg, n_responses=1)
        
    return response

In [ ]:
keys = ["mcq_4", "mcq_many", "frq"]
res = json.load(open("./results/final_results_conversation.json","r"))

res_with_exam["gpt-3.5"]["summarized"] = {}
res_with_exam["gpt-4"]["summarized"] = {}

gpt3_s_convos = pd.read_csv("./results/conversational_summarization_gpt3.tsv", sep="\t")
gpt4_s_convos = pd.read_csv("./results/conversational_summarization_gpt4.tsv", sep="\t")

n_trials = 10

for case_idx, case_id in tqdm(dataset.case_id.iteritems()):
    for key in keys:
        for j in range(n_trials):
            summarized_gpt3 = gpt3_s_convos.loc[case_idx, case_id]
            summarized_gpt4 = gpt4_s_convos.loc[case_idx, case_id]
            
            res_with_exam["gpt-3.5"]["summarized"][case] = {key:{"responses":None} for key in keys}
            res_with_exam["gpt-4"]["summarized"][case] = {key:{"responses":None} for key in keys}

            res_with_exam["gpt-3.5"]["summarized"][case][key]["responses"] = summarized_withPE_experiment(dataset, summarized_gpt3, case_idx, key, "gpt-3.5")
            res_with_exam["gpt-4"]["summarized"][case][key]["responses"] = summarized_withPE_experiment(dataset, summarized_gpt4, case_idx, key, "gpt-4")

# Evaluation

In [ ]:
evaluated_res = res_with_exam.copy()
for case in dataset.case_id:
    evaluated_res["gpt-3.5"]["summarized"][case] = eval_experiment(res_with_exam["gpt-3.5"]["summarized"][case],
                                                                   case_id = case,
                                                                   exp_keys = ["mcq_4", "mcq_many", "frq"],
                                                                   method = "autoeval",
                                                                   full_dataset = dataset,
                                                                   n_trials = 10)
    evaluated_res["gpt-4"]["summarized"][case] = eval_experiment(res_with_exam["gpt-4"]["summarized"][case],
                                                                 case_id = case,
                                                                 exp_keys = ["mcq_4", "mcq_many", "frq"],
                                                                 method = "autoeval",
                                                                 full_dataset = dataset,
                                                                 n_trials = 10)

In [ ]:
json.dump(evaluated_res, open("./results/final_results_conversation.json","w"))